In [1]:
# Check geemap installation
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package is not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

In [10]:
import ipywidgets
print (ipywidgets.__version__)

8.0.4


In [5]:
# Import libraries
import os
import ee
import geemap
import ipywidgets as widgets
from bqplot import pyplot as plt
from ipyleaflet import WidgetControl

In [6]:
# Create an interactive map
Map = geemap.Map(center=[40, -100], zoom=4, add_google_map=False)
Map.add_basemap('HYBRID')
Map.add_basemap('ROADMAP')

# Add Earth Engine data
fc = ee.FeatureCollection('TIGER/2018/Counties')
Map.addLayer(fc, {}, 'US Counties')

states = ee.FeatureCollection('TIGER/2018/States')
Map.addLayer(states, {}, 'US States')

        

Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [7]:
# Design interactive widgets

style = {'description_width': 'initial'}

output_widget = widgets.Output(layout={'border': '1px solid black'})
output_control = WidgetControl(widget=output_widget, position='bottomleft')
Map.add_control(output_control)

admin1_widget = widgets.Text(
    description='State:', value='Tennessee', width=200, style=style
)

admin2_widget = widgets.Text(
    description='County:', value='Knox', width=300, style=style
)

aoi_widget = widgets.Checkbox(
    value=False, description='Use user-drawn AOI', style=style
)

download_widget = widgets.Checkbox(
    value=False, description='Download chart data', style=style
)


def aoi_change(change):
    Map.layers = Map.layers[:4]
    Map.user_roi = None
    Map.user_rois = None
    Map.draw_count = 0
    admin1_widget.value = ''
    admin2_widget.value = ''
    output_widget.clear_output()

aoi_widget.observe(aoi_change, names='value')


year_widget = widgets.IntSlider(
    min=1984, max=2022, value=2022, description='Selected year:', width=400, style=style
)

nd_threshold = widgets.FloatSlider(
    value=0,
    min=-1,
    max=1,
    step=0.01,
    description='Threshold:',
    orientation='horizontal',
    style=style,
)


# Normalized Satellite Indices: https://www.usna.edu/Users/oceano/pguth/md_help/html/norm_sat.htm


# nd_indices.observe(nd_index_change, names='value')

submit = widgets.Button(
    description='Submit', button_style='primary', tooltip='Click me', style=style
)

full_widget = widgets.VBox(
    [
        widgets.HBox([admin1_widget, admin2_widget,year_widget]),
        widgets.HBox([aoi_widget, download_widget,nd_threshold]),
        submit,
    ]
)

full_widget

In [8]:
# Capture user interaction with the map


def handle_interaction(**kwargs):
    # retrive the coordinates after a click
    latlon = kwargs.get('coordinates')
    if kwargs.get('type') == 'click' and not aoi_widget.value:
        Map.default_style = {'cursor': 'wait'}
        xy = ee.Geometry.Point(latlon[::-1])
        selected_fc = fc.filterBounds(xy)

        with output_widget:
            output_widget.clear_output()

            
            # retrive the county and state name
            try:
                feature = selected_fc.first()
                admin2_id = feature.get('NAME').getInfo()
                statefp = feature.get('STATEFP')
                admin1_fc = ee.Feature(
                    states.filter(ee.Filter.eq('STATEFP', statefp)).first()
                )
                admin1_id = admin1_fc.get('NAME').getInfo()
                admin1_widget.value = admin1_id
                admin2_widget.value = admin2_id
                Map.layers = Map.layers[:4]
                geom = selected_fc.geometry()
                layer_name = admin1_id + '-' + admin2_id
                Map.addLayer(
                    ee.Image().paint(geom, 0, 2), {'palette': 'red'}, layer_name
                )
                print(layer_name)
            except:
                print('No feature could be found')
                Map.layers = Map.layers[:4]

        Map.default_style = {'cursor': 'pointer'}
    else:
        Map.draw_count = 0


Map.on_interaction(handle_interaction)

In [9]:
# Click event handler


def submit_clicked(b):

    with output_widget:
        output_widget.clear_output()
        print('Computing...')
        Map.default_style = {'cursor': 'wait'}

        
        # retrive the value from widget
        try:
            admin1_id = admin1_widget.value
            admin2_id = admin2_widget.value
            selected_year = year_widget.value
            use_aoi = aoi_widget.value
            threshold = nd_threshold.value


            if use_aoi:
                if Map.user_roi is not None:
                    roi = Map.user_roi
                    layer_name = 'User drawn AOI'
                    geom = roi
                else:
                    output_widget.clear_output()
                    print('No user AOI could be found.')
                    return
            else:

                # retrive the polygon (ROI)
                statefp = ee.Feature(
                    states.filter(ee.Filter.eq('NAME', admin1_id)).first()
                ).get('STATEFP')
                roi = fc.filter(
                    ee.Filter.And(
                        ee.Filter.eq('NAME', admin2_id),
                        ee.Filter.eq('STATEFP', statefp),
                    )
                )
                layer_name = admin1_id + '-' + admin2_id
                geom = roi.geometry()

            Map.layers = Map.layers[:4]     # maybe the US county
            Map.addLayer(ee.Image().paint(geom, 0, 2), {'palette': 'red'}, layer_name)

            
                     
           # Irrigation index added  
            ERA5_images_first = (ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY_BY_HOUR"))
            ERA5_images = ERA5_images_first.first()
            ERA5_image_irri = ERA5_images.clip(geom)

            irrigation = ERA5_image_irri.expression(
             'band1 - band2',
             { 
               'band1': ERA5_image_irri.select('total_precipitation'),
               'band2': ERA5_image_irri.select('total_evaporation')
             })

            #ERA5 bands, climate_image
#             Map.addLayer(ERA5_image_irri, {'bands': ['total_precipitation','total_evaporation']}, 'climate_image')

            # irrigation index
            Map.addLayer(irrigation, 
             {'min':-0.0001, 'max':0.0004,
             'palette': 
              [
                "#000080","#0000D9","#4000FF","#8000FF","#0080FF","#00FFFF",
                "#00FF80","#80FF00","#DAFF00","#FFFF00","#FFF500","#FFDA00",
                "#FFB000","#FFA400","#FF4F00","#FF2500","#FF0A00","#FF00FF",
                ]
             },
             'irrigation_index')
        
        
            # add colorbar
            colors = [
                "#000080","#0000D9","#4000FF","#8000FF","#0080FF","#00FFFF",
                "#00FF80","#80FF00","#DAFF00","#FFFF00","#FFF500","#FFDA00",
                "#FFB000","#FFA400","#FF4F00","#FF2500","#FF0A00","#FF00FF",
                ]
            vmin = -0.0001
            vmax = 0.0004

            Map.add_colorbar_branca(colors=colors, vmin=vmin, vmax=vmax, categorrical=True, step=4)
            Map.addLayerControl()
        
        except Exception as e:
            print(e)
            print('An error occurred during computation.')

        Map.default_style = {'cursor': 'default'}


submit.on_click(submit_clicked)